# Modeling Exercise: Logistic Regression
## Corey Solitaire
### 9.15.2020

In [4]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import acquire
import prepare
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# In this exercise, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

# For all of the models you create, choose a threshold that optimizes for accuracy.

In [6]:
# Cleaned Data for Exploration
df = acquire.get_titanic_data(cached = True)
train, validate, test = prepare.prep_titanic(df)

In [7]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embark_town,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
583,0,1,male,36.000000,0,0,40.1250,Cherbourg,1,1,0,0
337,1,1,female,41.000000,0,0,134.5000,Cherbourg,1,0,0,0
50,0,3,male,7.000000,4,1,39.6875,Southampton,0,1,0,1
218,1,1,female,32.000000,0,0,76.2917,Cherbourg,1,0,0,0
31,1,1,female,29.916875,1,0,146.5208,Cherbourg,0,0,0,0


### Inital Model:

### 1. Create another model that includes age in addition to fare and pclass. Does this model perform better than your previous one?

### 2. Include sex in your model as well. Note that you'll need to encode this feature before including it in a model.

### 3. Try out other combinations of features and models.

### 4. Choose you best model and evaluate it on the test dataset. Is it overfit?

### 5. Bonus How do different strategies for handling the missing values in the age column affect model performance?

### 6. Bonus: How do different strategies for encoding sex affect model performance?

### 7. Bonus: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.

### Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.

### C=.01,.1,1,10,100,1000

### Bonus Bonus: how does scaling the data interact with your choice of C?